In [2]:
from pathlib import Path
import random
from collections import defaultdict
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

%config InlineBackend.figure_format = 'retina'

plt.style.use('seaborn-paper')
mpl.rcParams['axes.titlesize'] = 18
mpl.rcParams['font.family'] = ['serif']
mpl.rcParams['text.usetex'] = True
mpl.rcParams['font.serif'] = ['Computer Modern Roman']
mpl.rcParams['axes.labelsize'] = 14
mpl.rcParams['legend.fontsize'] = 14
mpl.rcParams['xtick.labelsize'] = 14
mpl.rcParams['ytick.labelsize'] = 14

from crawlercrawler import load_zip_json

# GPT2 Data schema
  * `List[method_results]`
  * `method_results = ['method_signature', 'hypothesis_docstring', 'target_docstrings']`

In [3]:
MODELDIR = Path('/smartml-athena/processed-data/fundef-docstrings/models/gpt2/')
GPT2_NO_DROPOUT = MODELDIR / 'expt-2020-02-17/2020-02-20/checkpoint-37400/python-func-def-docstrings-2020-02-15-2252-3165559177.test.predictions.json.gz'
GPT2_DROPOUT = MODELDIR / 'expt-2020-02-18/2020-02-21/checkpoint-44000/python-func-def-docstrings-2020-02-15-2252-3165559177.test.predictions.json.gz'

In [4]:
def normalize_gpt2(example):
    return [
            example[0], 
            example[1].replace('"""', '').strip(),
            example[2].replace('"""', '').strip()
    ]

In [5]:
dat_gpt2_dropout = list(map(normalize_gpt2, load_zip_json(GPT2_DROPOUT)))
dat_gpt2_no_dropout = list(map(normalize_gpt2, load_zip_json(GPT2_NO_DROPOUT)))

In [6]:
same = 0
for d1, d2 in zip(dat_gpt2_dropout, dat_gpt2_no_dropout):
    if d1[0] == d2[0]:
        same += 1
print(same)
print(len(dat_gpt2_dropout))

342861
342861


In [7]:
print(dat_gpt2_dropout[0])
print(dat_gpt2_no_dropout[0])

['def main():\n', 'Main function', 'docstring for main']
['def main():\n', 'Main function', 'docstring for main']


# Fairseq Data schema
  * More complex than above, but has the same 3 features for each example, indicated by the first token on each line
    - `S-104` -- Source 104 (method example 104 given as source for translation
    - `H-104` -- Hypothesis 104 (generated docstring)
    - `T-104` -- Target 104 (target 'true' docstring)

In [8]:
FAIRSEQ_MODELDIR = Path('/smartml-athena/processed-data/fundef-docstrings/split/translate-method-docstring/fairseq')
TRANSLATION = FAIRSEQ_MODELDIR / '2020-02-25/generated/generate-test.txt'
PRETRAINED_TRANSLATION = FAIRSEQ_MODELDIR / '2020-03-29-pretrained/generated/generate-test.txt'
PRETRAINED_EARLY_TRANSLATION = FAIRSEQ_MODELDIR / '2020-03-29-pretrained/generated-checkpoint153/generate-test.txt'

In [9]:
REPLACE = {'Ġ': ' ', 'Ċ': '\n', 'ĉ': '\t', 'NEWLINE': '\n', '<user_handle>': 'param'}

def fix_whitespace(string):
    """Return tokenized data to string literal"""
    for r, f in REPLACE.items():
        string = string.replace(r, f)
    return string

def decode(outlines):
    """Reads gathered fairseq-generate lines, returns src, hyp, tgt"""
    return [
            fix_whitespace(''.join(outlines[0][1:])),
            fix_whitespace(''.join(outlines[1][1:])),
            fix_whitespace(''.join(outlines[2][2:]))
    ]

def put_together():
    assert False
    splitsrc = src.split('):\n')
    sig, body = splitsrc[0] + '):\n', '):\n'.join(splitsrc[1:])
    
    target = sig + '    """\n{}\n    """\n'.format(indent(tgt, ' '* 4)) + body
    hypothesis = sig + '    """\n{}\n    """\n'.format(indent(hyp, ' '* 4)) + body
    return [src, hypothesis, target]

def normalize_fairseq_schema(filepath):
    """Reads fairseq generated file and outputs results in the form of the GPT2 schema above"""
    gathered = defaultdict(list)
    with Path(filepath).open() as fin:
        for line in fin.readlines():
            if line[0] in ('S', 'T', 'H', 'P'):
                tokens = line.split()
                gathered[int(tokens[0].split('-')[1])].append(tokens)
    return [decode(gathered[k]) for k in gathered]

In [10]:
dat_translation = normalize_fairseq_schema(TRANSLATION)
dat_pretrained_translation = normalize_fairseq_schema(PRETRAINED_TRANSLATION)
dat_pretrained_early_translation = normalize_fairseq_schema(PRETRAINED_EARLY_TRANSLATION)

# Lining up generated examples

The different frameworks do not guarantee alignment of examples or even a one-to-one matchup of examples. Here I'll try to find all the examples which co-occur, and align them for each dataset. I will extract the text of the docstrings from the GPT2 data, match it in the fairseq data, and then match for method signature.

In [46]:
dat_gpt2_dropout[0][2].replace('"""', '').strip()

'docstring for main'

In [164]:
gpt2_dat_dict = {
    d[2]: i for 
    i, d in enumerate(dat_gpt2_dropout)
}

In [50]:
t = hash('docstring for main')

In [165]:
translation_gpt2_aligned = [] 
candidates = []
for j, d in enumerate(dat_translation):
    if d[2].strip() in gpt2_dat_dict:
        i = gpt2_dat_dict[d[2].strip()]
        if dat_gpt2_dropout[i][0].strip() in d[0]:
            translation_gpt2_aligned.append([j, i])
        else:
            candidates.append([j,i])

In [166]:
print(len(translation_gpt2_aligned)/len(dat_gpt2_dropout))

0.05314690209735141


Only 5% of the docstrings in fairseq test data can be positively matched to gpt2 test data!

In [92]:
i, j = translation_gpt2_aligned[12000]
print(dat_translation[i][0], '\n', dat_translation[i][2])
print(dat_gpt2_dropout[j][0], '\n', dat_gpt2_dropout[j][2])

def init_logstart(self):
    if self.logappend:
        self.magic(('logstart %s append' % self.logappend))
    elif self.logfile:
        self.magic(('logstart %s' % self.logfile))
    elif self.logstart:
        self.magic('logstart') 
 Initialize logging in case it was requested at the command line.
        
def init_logstart(self):
 
     """    Initialize logging in case it was requested at the command line.
    """



In [82]:
len(candidates)

7198

In [90]:
i, j = candidates[2000]
print(dat_translation[i][0], '\n', dat_translation[i][2])
print(dat_gpt2_dropout[j][0], '\n', dat_gpt2_dropout[j][2])

@since('1.6.0')
def setMinTF(self, value):
    return self._set(minTF=value) 
 Sets the value of :py:attr:.
@since('2.0.0')
def setMinDivisibleClusterSize(self, value):
 
     """
    Sets the value of :py:attr:.
    """



In [87]:
[print(dat_translation[i][0])]

@property
def created_at(self):
    return self._created_at.value


In [88]:
print(dat_translation[i][2])

:type: datetime.datetime


In [11]:
from nltk.translate import bleu, bleu_score

def get_bleu(dat, head=None, thin=None):
    refs, hyps = [], []
    if head is not None:
        dat = dat[:head]
    if thin is not None:
        dat = dat[::thin]
    for d in dat:
        r, h = d[2].split(), d[1].split()
        if len(r) > 4 and len(h) > 4:
            refs.append([r])
            hyps.append(h)

    return bleu_score.corpus_bleu(refs, hyps)

In [12]:
get_bleu(dat_pretrained_translation)

0.3250250849131691

In [13]:
get_bleu(dat_translation)

0.35805601517315067

In [14]:
get_bleu(dat_gpt2_dropout)

0.11053384379722456

In [15]:
get_bleu(dat_gpt2_no_dropout)

0.11003575529065572

In [504]:
get_bleu(dat_pretrained_early_translation)

0.31049806007581465

In [21]:
from multiprocessing import Pool
from collections import defaultdict

In [17]:
def get_dat_folds(dat, size, number, maxlen=None, seed=98115):
    if maxlen is not None:
        dat = dat[:maxlen]  # for consistent shuffle

    dat_filtered = [d for d in dat if len(d[1]) > 0 and len(d[2])> 0]
    order = np.arange(len(dat_filtered))
    rng = np.random.RandomState(1)
    rng.shuffle(order)
    
    folds = [order[i * size: (i + 1) * size] for i in range(number)]
    foldlist = []
    for fold in folds:
        hyp, ref = [], []
        for i in fold:
            if len(dat_filtered[i][1]) > 0 and len(dat_filtered[i][2])> 0:
                hyp.append(dat_filtered[i][1])
                ref.append(dat_filtered[i][2])
        foldlist.append([hyp, ref])
    return foldlist

datasets = {
    'gpt2': dat_gpt2_no_dropout,
    #'gpt2_dropout': dat_gpt2_dropout,
    'translation': dat_translation,
    'pretrained_translation': dat_pretrained_translation
}

maxlen = min([len(d) for d in datasets.values()])


fold_list = []
for label, dat in datasets.items():
    for fold in get_dat_folds(dat, 1024, 12, maxlen=maxlen):
        fold_list.append((label, fold))

In [18]:
from rouge import Rouge

def get_rouge(fold):
    label, (hyp, ref) = fold
    return fold[0], Rouge().get_scores('\n'.join(hyp), '\n'.join(ref))[0]

In [19]:
def flatten_dict(results):
    flatdict = {}
    for metric in ('rouge-1', 'rouge-2', 'rouge-l'):
        flatdict[metric] = {}
        for stat in ('f', 'p', 'r'):
            flatdict[metric][stat] = []
            for r in results:
                flatdict[metric][stat].append(r[metric][stat])
    return flatdict

In [22]:
pool = Pool()

results_dict = defaultdict(list)
for r in pool.map(get_rouge, fold_list):
    results_dict[r[0]].append(r[1])

for model in results_dict:
    results_dict[model] = flatten_dict(results_dict[model])

In [23]:
results_dict

defaultdict(list,
            {'gpt2': {'rouge-1': {'f': [0.4150783518558499,
                0.409952805199533,
                0.4222222174998979,
                0.4380468975576276,
                0.431306201209186,
                0.4326822415031099,
                0.4394779140384913,
                0.46291300938930396,
                0.42361408141206225,
                0.4089358752365651,
                0.45665712366749384,
                0.4240357240762593],
               'p': [0.5275624461670974,
                0.5270240458857269,
                0.5524002704530088,
                0.5518077601410935,
                0.5296304395364093,
                0.5581552305961754,
                0.5532297996848976,
                0.5888912717134892,
                0.5346020761245674,
                0.5389975275342773,
                0.5840688912809473,
                0.5586221651690201],
               'r': [0.3421309872922776,
                0.3354394832912103,
         

In [27]:
results_dict.keys()

dict_keys(['gpt2', 'translation', 'pretrained_translation'])

In [28]:
def mean_std_dict(results):
    meandict = {}
    stddict = {}
    for metric in results:
        meandict[metric] = {}
        stddict[metric] = {}
        for stat in results[metric]:
            meandict[metric][stat] = np.mean(results[metric][stat])
            stddict[metric][stat] = np.std(results[metric][stat])
    return meandict, stddict

In [29]:
for model in results_dict:
    means, stds = mean_std_dict(results_dict[model])
    print(f'{model}:')
    for metric in means:
        print(f'\t{metric}:')
        for stat in means[metric]:
            m, s = means[metric][stat], stds[metric][stat]
            print(f'\t\t{stat} = {m:.4f} +/- {s:.4f}')

gpt2:
	rouge-1:
		f = 0.4304 +/- 0.0162
		p = 0.5504 +/- 0.0197
		r = 0.3535 +/- 0.0155
	rouge-2:
		f = 0.2109 +/- 0.0164
		p = 0.2573 +/- 0.0204
		r = 0.1788 +/- 0.0143
	rouge-l:
		f = 0.3852 +/- 0.0167
		p = 0.5373 +/- 0.0193
		r = 0.3451 +/- 0.0152
translation:
	rouge-1:
		f = 0.6157 +/- 0.0139
		p = 0.5319 +/- 0.0178
		r = 0.7313 +/- 0.0104
	rouge-2:
		f = 0.4410 +/- 0.0148
		p = 0.3791 +/- 0.0162
		r = 0.5272 +/- 0.0129
	rouge-l:
		f = 0.5796 +/- 0.0172
		p = 0.5252 +/- 0.0176
		r = 0.7220 +/- 0.0101
pretrained_translation:
	rouge-1:
		f = 0.6056 +/- 0.0131
		p = 0.5190 +/- 0.0148
		r = 0.7274 +/- 0.0133
	rouge-2:
		f = 0.4193 +/- 0.0205
		p = 0.3560 +/- 0.0189
		r = 0.5103 +/- 0.0238
	rouge-l:
		f = 0.5660 +/- 0.0158
		p = 0.5114 +/- 0.0152
		r = 0.7168 +/- 0.0140


## gpt2:
  * bleu: 0.110
  * rouge-1:
    - f = 0.4351 +/- 0.0067
	- p = 0.5526 +/- 0.0104
	- r = 0.3589 +/- 0.0065
  * rouge-2:
	- f = 0.2264 +/- 0.0068
	- p = 0.2786 +/- 0.0082
	- r = 0.1908 +/- 0.0064
  * rouge-l:
	- f = 0.3911 +/- 0.0070
	- p = 0.5396 +/- 0.0110
	- r = 0.3504 +/- 0.0064
 
## gpt2_dropout:
  * bleu: 0.110
  * rouge-1:
	- f = 0.4272 +/- 0.0089
	- p = 0.5568 +/- 0.0077
	- r = 0.3467 +/- 0.0105
  * rouge-2:
	- f = 0.2239 +/- 0.0091
	- p = 0.2883 +/- 0.0093
	- r = 0.1831 +/- 0.0091
  * rouge-l:
	- f = 0.3781 +/- 0.0114
	- p = 0.5433 +/- 0.0075
	- r = 0.3383 +/- 0.0104

## translation:
  * bleu: 0.358
  * rouge-1:
	- f = 0.6098 +/- 0.0094
	- p = 0.5240 +/- 0.0113
	- r = 0.7295 +/- 0.0070
  * rouge-2:
	- f = 0.4419 +/- 0.0111
	- p = 0.3771 +/- 0.0114
	- r = 0.5337 +/- 0.0110
  * rouge-l:
	- f = 0.5719 +/- 0.0114
	- p = 0.5171 +/- 0.0114
	- r = 0.7199 +/- 0.0072
  
## pretrained_translation:
  * bleu: 0.325
  * rouge-1:
    - f = 0.5985 +/- 0.0085
	- p = 0.5111 +/- 0.0113
	- r = 0.7222 +/- 0.0095
  * rouge-2:
	- f = 0.4209 +/- 0.0091
	- p = 0.3546 +/- 0.0108
	- r = 0.5181 +/- 0.0081
  * rouge-l:
	- f = 0.5577 +/- 0.0116
	- p = 0.5034 +/- 0.0117
	- r = 0.7113 +/- 0.0095


In [364]:
from fairseq.models.transformer import TransformerModel
from tokenizers import ByteLevelBPETokenizer

In [505]:
ls /smartml-athena/processed-data/fundef-docstrings/split/translate-method-docstring

data-bin/
dawn-fairseq/
dawn-fairseq-javadoc-problems/
fairseq/
python-func-def-docstrings-2020-02-21-2026_bytelevelbpe_30000-merges.txt
python-func-def-docstrings-2020-02-21-2026_bytelevelbpe_30000-vocab.json
python-func-def-docstrings-2020-02-21-2026.test.docstring.json.gz
python-func-def-docstrings-2020-02-21-2026.test.methods.json.gz
python-func-def-docstrings-2020-02-21-2026.train.docstring.json.gz
python-func-def-docstrings-2020-02-21-2026.train.methods.json.gz
python-func-def-docstrings-2020-02-21-2026.val.docstring.json.gz
python-func-def-docstrings-2020-02-21-2026.val.methods.json.gz
python-func-def-docstrings-2020-03-04-2336.test.docstring.json.gz
python-func-def-docstrings-2020-03-04-2336.test.methods.json.gz
python-func-def-docstrings-2020-03-04-2336.train.docstring.json.gz
python-func-def-docstrings-2020-03-04-2336.train.methods.json.gz
python-func-def-docstrings-2020-03-04-2336.val.docstring.json.gz
python-func-def-docstrings-2020-03-04-2336.val.methods.json.gz


In [507]:
ROOT = Path('/smartml-athena/processed-data/fundef-docstrings/split/translate-method-docstring')
MODEL_ROOT = ROOT / 'fairseq/2020-02-25/'
MODEL_CHECKPOINT = MODEL_ROOT / 'checkpoint_best.pt'
DATADIR = '/smartml-athena/processed-data/fundef-docstrings/split/translate-method-docstring/fairseq/binary_javadoc_problems'

# TOKENIZER_ROOT = Path('/smartml-athena/processed-data/fundef-docstrings/dawn-py-pretrained/forColin')
# VOCAB_FILE = str(TOKENIZER_ROOT / 'hacky_byte_level-vocab.json')
# MERGES_FILE = str(TOKENIZER_ROOT / 'hacky_byte_level-merges.txt')

TOKENIZER_PREFIX = 'python-func-def-docstrings-2020-02-21-2026_bytelevelbpe_30000-'
VOCAB_FILE = str(ROOT / (TOKENIZER_PREFIX + 'vocab.json'))
MERGES_FILE = str(ROOT / (TOKENIZER_PREFIX + 'merges.txt'))

tokenizer = ByteLevelBPETokenizer(VOCAB_FILE, MERGES_FILE)


In [570]:
model = TransformerModel.from_pretrained(
    MODEL_ROOT,
    checkpoint_file='checkpoint_best.pt',
    data_name_or_path=DATADIR,
)

model.to('cuda')

2020-04-07 20:29:09,039 - fairseq.file_utils - INFO: loading archive file /smartml-athena/processed-data/fundef-docstrings/split/translate-method-docstring/fairseq/2020-02-25 from cache at /smartml-athena/processed-data/fundef-docstrings/split/translate-method-docstring/fairseq/2020-02-25
2020-04-07 20:29:09,040 - fairseq.file_utils - INFO: loading archive file /smartml-athena/processed-data/fundef-docstrings/split/translate-method-docstring/fairseq/binary_javadoc_problems
2020-04-07 20:29:11,823 - fairseq.tasks.translation - INFO: [methods] dictionary: 29936 types
2020-04-07 20:29:11,824 - fairseq.tasks.translation - INFO: [docstring] dictionary: 29936 types
2020-04-07 20:29:17,277 - fairseq.models.fairseq_model - INFO: Namespace(activation_dropout=0.2, activation_fn='relu', adam_betas='(0.9,0.98)', adam_eps=1e-06, adaptive_input=False, adaptive_softmax_cutoff=None, adaptive_softmax_dropout=0, all_gather_list_size=16384, arch='transformer', attention_dropout=0.1, best_checkpoint_metri

GeneratorHubInterface(
  (models): ModuleList(
    (0): TransformerModel(
      (encoder): TransformerEncoder(
        (embed_tokens): Embedding(29936, 1472, padding_idx=1)
        (embed_positions): SinusoidalPositionalEmbedding()
        (layers): ModuleList(
          (0): TransformerEncoderLayer(
            (self_attn): MultiheadAttention(
              (k_proj): Linear(in_features=1472, out_features=1472, bias=True)
              (v_proj): Linear(in_features=1472, out_features=1472, bias=True)
              (q_proj): Linear(in_features=1472, out_features=1472, bias=True)
              (out_proj): Linear(in_features=1472, out_features=1472, bias=True)
            )
            (self_attn_layer_norm): _FusedLayerNorm(torch.Size([1472]), eps=1e-05, elementwise_affine=True)
            (fc1): Linear(in_features=1472, out_features=4096, bias=True)
            (fc2): Linear(in_features=4096, out_features=1472, bias=True)
            (final_layer_norm): _FusedLayerNorm(torch.Size([1472]

In [523]:
from fairseq.search import Sampling

In [569]:
TransformerModel.from_pretrained?

In [575]:
def translate(plain_text, tokenizer=tokenizer, model=model):
    encoding = ' '.join(tokenizer.encode(plain_text).tokens)
    return fix_whitespace(''.join(model.translate(
        encoding, beam=5, temperature=1.).split()))

In [576]:
print(dat_translation[247102][0])
print('\n')
print(translate(dat_translation[247102][0]))
print(dat_translation[247102][2])

def add_insect(self, insect):
    if insect.is_ant:
        if (self.ant is None):
            self.ant = insect
        elif self.ant.can_contain(insect):
            self.ant.ant = insect
        elif insect.can_contain(self.ant):
            insect.ant = self.ant
            self.ant = insect
        else:
            assert (self.ant is None), 'Two ants in {0}'.format(self)
    else:
        self.bees.append(insect)
    insect.place = self


Add an INSECT to this Place.
Add an INSECT to this Place.


In [564]:
print(translate(dat_translation[121582][0]))

isShapeInsideOfBox(shapeCoordinates, boxCoordinates) -> True if
shapeCoordinates is inside the box, False otherwise.


In [443]:
key = random.randint(0, len(dat_translation))
print(key)
print(dat_translation[key][0])
print(dat_translation[key][1])

121582
def isShapeInsideOfBox(self, shapeCoordinates, boxCoordinates):
    for point in shapeCoordinates:
        if (not self.isPointInsideOfBox(point, boxCoordinates)):
            return False
    return True
Go through every point in the shape to test if they are all inside the box.


In [404]:
print(dat_pretrained_translation[key][0])
print(dat_pretrained_translation[key][1])

def handle_paragraph(self, tag):
    name = tag.name
    print(tag)
    if tag.find('label'):
        print(tag)
        label = tag.split('label ')[(- 1)]
        return '\n\\begin{{{name}}} \\label{{{label}}}\n    {content}\n\\end{{{name}}}\n'.format(name='subsection', content=self.format(tag), label=label)
    else:
        return '\n\\begin{{{name}}}\n    {content}\n\\end{{{name}}}\n'.format(name=self.tag_to_latex[name], content=self.format(tag))
Parameters
----------
tag: QqTag

Returns "egin{subsection}
            tag content
         \end{subsection}
-------


In [400]:
random.randint(0, len(dat_translation))

111844